In [1]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import re
import spacy
from transformers import pipeline
import seaborn as sns
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from torch import cuda
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import BCEWithLogitsLoss
import nltk
from transformers import Trainer
nltk.download('words')
from nltk.corpus import words
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments
from collections import Counter
import torch.optim as optim
import os
from nltk.stem.snowball import SnowballStemmer
import spacy
from tqdm import tqdm


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [346]:
df_train = pd.read_csv('training_preprocesat.csv')


KeyboardInterrupt



In [ ]:
len(df_train)

In [2]:
tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})

robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
robert.resize_token_embeddings(len(tokenizer))

Embedding(50005, 768)

In [3]:
df_shuffled = df_train.sample(frac=0.2, random_state=1000)
train_ratio = 0.8
train_size = int(train_ratio * len(df_shuffled))
df_train = df_shuffled[:train_size]
df_val = df_shuffled[train_size:]

NameError: name 'df_train' is not defined

In [ ]:
X_train = []
y_train = []
aux = list(zip(df_train['title'].tolist(), df_train['class'].tolist()))

In [ ]:
for title, label in tqdm(aux):
    input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
    outputs = robert(input_ids)
    mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
    X_train.append(mean_pooling)
    y_train.append(label)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_val = []
y_val = []
aux = list(zip(df_val['title'].tolist(), df_val['class'].tolist()))

In [ ]:
np.save('x_train.npy', X_train)
np.save('y_train.npy', y_train)
np.save('x_val.npy', X_val)
np.save('y_val.npy', y_val)

In [ ]:
for title, label in tqdm(aux):
    input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
    outputs = robert(input_ids)
    mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
    X_val.append(mean_pooling)
    y_val.append(label)

In [ ]:
X_val = np.array(X_val)
y_val = np.array(y_val)

In [ ]:
# AICI INCEPE MODDELUL PENTRU TITLU

In [4]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return torch.tensor(self.X[index]), torch.tensor(self.y[index])

In [5]:
class ClssificationNetwork(nn.Module):
    def __init__(self):
        super(ClssificationNetwork, self).__init__()
        self.linear = nn.Linear(768, 64)
        self.fc = nn.Linear(64, 64)
        self.bn = nn.BatchNorm1d(64) 
        self.classifier = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.fc(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.classifier(x)
        x = self.sigmoid(x)

        return x

In [6]:
train_set = CustomDataset(X_train, y_train)
val_set = CustomDataset(X_val, y_val)

train_loader = DataLoader(train_set, shuffle=True, batch_size=16)
val_loader = DataLoader(val_set, shuffle=True, batch_size=16)

NameError: name 'X_train' is not defined

In [7]:
def get_training_data():
    first = pd.read_csv('first.csv')
    first_titles = np.load('training_splits/firsttitle.npy')
    first_labels = first['class'].tolist()

    second = pd.read_csv('second.csv')
    second_titles = np.load('training_splits/secondtitle.npy')
    second_labels = second['class'].tolist()

    third = pd.read_csv('third.csv')
    third_titles = np.load('training_splits/thirdtitle.npy')
    third_labels = third['class'].tolist()

    fourth = pd.read_csv('fourth.csv')
    fourth_titles = np.load('training_splits/fourthtitle.npy')
    fourth_labels = fourth['class'].tolist()

    all_titles = np.concatenate((first_titles, second_titles, third_titles, fourth_titles), axis=0)
    all_labels = np.concatenate((first_labels, second_labels, third_labels, fourth_labels))
    return all_titles, all_labels

def get_validation_data():
    fourth = pd.read_csv('fourth.csv')
    fourth_titles = np.load('training_splits/fourthtitle.npy')
    fourth_labels = fourth['class'].tolist()
    
    return fourth_titles, fourth_labels

In [8]:
X_train, y_train = get_training_data()
# X_val, y_val = get_validation_data()

In [9]:
train_set = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_set, shuffle=True, batch_size=128)

# val_set = CustomDataset(X_val, y_val)
# val_loader = DataLoader(val_set, shuffle=True, batch_size=16)

In [12]:
model_titles = ClssificationNetwork().to(device)
optimizer = optim.Adam(model_titles.parameters(),lr=1e-4)
criterion = nn.BCELoss(weight=torch.tensor([len(y_train) / np.sum(y_train)]).to(device))

In [13]:
EPOCHS = 50

In [14]:
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm

for epoch in range(EPOCHS):
    running_loss = 0
    running_acc = 0

    train_pred_labels = []
    train_true_labels = []

    for X, y in tqdm(train_loader):
        y = y.to(device, dtype=torch.long)
        X = X.to(device, dtype=torch.float)

        output = model_titles(X)

        output.squeeze(1)
        pred = output > 0.5

        train_pred_labels.extend(pred.cpu().numpy().tolist())
        train_true_labels.extend(y.cpu().numpy().tolist())
        
        loss = criterion(output.squeeze(1), y.float())
        running_loss += loss.item()

        model_titles.zero_grad()
        loss.backward()
        optimizer.step()

    # pred_labels = []
    # true_labels = []
    # 
    # model_titles.eval()
    # val_loss = 0
    # with torch.no_grad():
    #     for X, y in tqdm(val_loader):
    #         y = y.to(device, dtype=torch.long)
    #         X = X.to(device, dtype=torch.float)
    # 
    #         output = model_titles(X)
    #         val_loss += criterion(output.squeeze(1), y.float()).item()
    #         output.squeeze(1)
    # 
    #         pred = (output > 0.5)
    #         pred_labels.extend(pred.cpu().numpy().tolist())
    #         true_labels.extend(y.cpu().numpy().tolist())
    # 
    # model_titles.train()
    # 
    # precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='macro')
    precision_train, recall_train, f1_train, _ = precision_recall_fscore_support(train_pred_labels, train_true_labels, average='macro')

    print(f"Epoch: {epoch + 1} | Train Loss {running_loss / len(train_set)}  | Train Precision: {precision_train} | Train Recall: {recall_train} | Train F1: {f1_train}")
    # print(f"Epoch: {epoch + 1} | Train Loss {running_loss / len(train_set)}  | Train Precision: {precision_train} | Train Recall: {recall_train} | Train F1: {f1_train} | Val Loss: {val_loss / len(val_set)} | Val Precision: {precision} | Val Recall: {recall} | Val F1: {f1}")

torch.save(model_titles.state_dict(), 'models/titleClassifier.pth')

  0%|          | 0/552 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_11296\2081287659.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.X[index]), torch.tensor(self.y[index])
100%|██████████| 552/552 [00:01<00:00, 398.29it/s]


Epoch: 1 | Train Loss 0.003515754671859098  | Train Precision: 0.9035580109285697 | Train Recall: 0.8958046848060617 | Train F1: 0.899365709648426


  0%|          | 0/552 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_11296\2081287659.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.X[index]), torch.tensor(self.y[index])
100%|██████████| 552/552 [00:01<00:00, 454.69it/s]


Epoch: 2 | Train Loss 0.002039492212838545  | Train Precision: 0.9395799670622464 | Train Recall: 0.9424370055123696 | Train F1: 0.9409765615121961


  0%|          | 0/552 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_11296\2081287659.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.X[index]), torch.tensor(self.y[index])
100%|██████████| 552/552 [00:01<00:00, 495.73it/s]


...

Epoch: 48 | Train Loss 0.00027779832349103344  | Train Precision: 0.9915776314794236 | Train Recall: 0.9924572542229129 | Train F1: 0.9920147952422398


  0%|          | 0/552 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_11296\2081287659.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.X[index]), torch.tensor(self.y[index])
100%|██████████| 552/552 [00:02<00:00, 231.93it/s]


Epoch: 49 | Train Loss 0.00028596991769883843  | Train Precision: 0.990905252918659 | Train Recall: 0.9919573305467362 | Train F1: 0.9914275047454417


  0%|          | 0/552 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_11296\2081287659.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.X[index]), torch.tensor(self.y[index])
100%|██████████| 552/552 [00:02<00:00, 216.93it/s]


Epoch: 50 | Train Loss 0.00027392673519321384  | Train Precision: 0.9912940239337573 | Train Recall: 0.9922165442920534 | Train F1: 0.9917523710963201


In [ ]:
# GATA MODELULU TITLU
# GATA MODELULU TITLU

In [ ]:
def test_embedding():
    df = pd.read_csv('test_preprocesat.csv')
    X = []
    
    for title in tqdm(df['title']):
        input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
        outputs = robert(input_ids)
        mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
        X.append(mean_pooling)
        
    X = np.array(X)
    np.save('test_embeddings.npy', X)
    return X

In [ ]:
# X_test = test_embedding()
X_test = np.load('test_embeddings.npy')

In [ ]:
X_test = np.load('test_embeddings.npy')

In [ ]:
model = ClssificationNetwork().to(device)
model.load_state_dict(torch.load('haisavedem.pth'))

In [ ]:
class CustomTestDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return torch.tensor(self.X[index]), y

In [ ]:
model.eval()
predictions = []
indices = []
cnt = 0
with torch.no_grad():
    for i in tqdm(range(len(X_test))):
        axu = torch.tensor(X_test[i], dtype=torch.float).to(device)
        prediction = model(axu) > 0.5
        predictions.append(prediction.item())
        indices.append(cnt)
        cnt += 1
model.train()

In [ ]:
len(ids)

In [ ]:
submisie = pd.DataFrame({'id':indices, 'class':list(map(int, predictions))})
submisie.to_csv('submission.csv', index=False)

In [ ]:
# preprocess the data for the model
def replace_unwanted_characters(df):
    df['content'] = df['content'].fillna('')
    df['title'] = df['title'].fillna('')
    df['title'] = df['title'].apply(lambda x: x.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș"))
    df['content'] = df['content'].apply(lambda x: x.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș"))
    return df

repeated_characters = [x * 3 for x in "AĂÂBCDEFGHÎJKLMNOPQRSȘTȚUVWXYZaăâbcdefghîjklmnopqrsștțuvwxyz,.-/';[]-!@#$%^&*()?+"]
repeated_characters.append('iiii')
repeated_characters.append('IIII')

profanity_list = [
    "muie",
    "laba",
    "labă",
    "pula",
    "pulă",
    "pizda",
    "pizdă",
    "ce pula",
    "ce pulă",
    "ce pizda",
    "ce pizdă",
    "caca",
    "cacat",
    "căcat",
    "pipi",
    "pisat",
    "pișat",
    "pishat",
    "rahat",
    "kkt",
    "kk",
    "plm",
    "ma fut",
    "mă fut",
    "ma cac",
    "mă cac",
    "ma pis",
    "mă pis",
    "ma pish",
    "mă pish",
    "pwla",
    "pwlă",
    "p.u.l.a.",
    "poola",
    "naiba",
    "dracu",
    "draq",
    "drecu",
    "naibii",
    "dracului",
    "drecului",
    "drqlui",
    "coaie",
    "coae",
    "sloboz",
    "lindic",
    "gaoz",
    "ochiul maro",
    "floci",
    "cur",
    "futai",
    "futare",
    "futere",
    "popou",
    "nanau",
    "pulii",
    "pulii mele",
    "coaiele",
    "coaiele mele",
    "pulile",
    "măta",
    "mata",
    "mă-tii",
    "mă-ta",
    "mă-ti",
]

english_words = set(words.words())

def normalize_text(df):

    df['title'] = df['title'].apply(lambda x : re.sub(r"<.*?>", '', x ))
    df['content'] = df['content'].apply(lambda x:re.sub(r"<.*?>", '', x))

    df['title'] = df['title'].apply(lambda x : re.sub(r"\.\.\.", '', x ))
    df['content'] = df['content'].apply(lambda x:re.sub(r"\.\.\.", '', x))

    df['title'] = df['title'].apply(lambda x : re.sub(r"#[\w+-]+", '[HTAG]', x ))
    df['content'] = df['content'].apply(lambda x:re.sub(r"#[\w+-]+", '[HTAG]', x))

    df['title'] = df['title'].apply(lambda x : ' '.join(["[STAR]" if "*" in word else word for word in x.split()]))
    df['content'] = df['content'].apply(lambda x:' '.join(["[STAR]" if "*" in word else word for word in x.split()]))

    df['title'] = df['title'].apply(lambda x : ' '.join(["[REP]" if any([rep in word for rep in repeated_characters]) else word for word in x.split()]))
    df['content'] = df['content'].apply(lambda x : ' '.join(["[REP]" if any([rep in word for rep in repeated_characters]) else word for word in x.split()]))

    pattern = r'^[!?"\',“”-]*(.*?)[!?"\',“”-]*$'
    df['title'] = df['title'].apply(lambda x : ' '.join(["[PROF]" if re.sub(pattern, r'\1', word) in profanity_list else word for word in x.split()]))
    df['content'] = df['content'].apply(lambda x : ' '.join(["[PROF]" if re.sub(pattern, r'\1', word) in profanity_list else word for word in x.split()]))


    emoji_pattern = r"""
                    (?:
                      [<>]?
                      [:;=8]                     # eyes
                      [\-o\*\']?                 # optional nose
                      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
                      |
                      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
                      [\-o\*\']?                 # optional nose
                      [:;=8]                     # eyes
                      [<>]?
                      |
                      </?3                       # heart
                    )"""
    df['title'] = df['title'].apply(lambda x : re.sub(emoji_pattern, '[EMOJI]', x ))
    df['content'] = df['content'].apply(lambda x : re.sub(emoji_pattern, '[EMOJI]', x ))


    return df

In [ ]:
def split_train_in_four(dataset):
    length = int(len(dataset) / 4)
    
    first = dataset[:length]
    second = dataset[length:2*length]
    third = dataset[2*length:3*length]
    fourth = dataset[3*length:4*length]

    first.to_csv('first.csv', encoding='utf-8', index=False)
    second.to_csv('second.csv', encoding='utf-8', index=False)
    third.to_csv('third.csv', encoding='utf-8', index=False)
    fourth.to_csv('fourth.csv', encoding='utf-8', index=False)
    
split_train_in_four(pd.read_csv('data/train.csv'))

In [ ]:
def get_titles_tokens(df):
    X_train = []
    y_train = []
    aux = list(zip(df['title'].tolist(), df['class'].tolist()))

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})
    
    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))

    for title, label in tqdm(aux):
        input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
        outputs = robert(input_ids)
        mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
        X_train.append(mean_pooling)
        y_train.append(label)

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    return X_train, y_train

def get_content_tokens(df):
    X_train = []
    y_train = []
    aux = list(zip(df['content'].tolist(), df['class'].tolist()))

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})
    
    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))
        
    for content, label in tqdm(aux):
        if len(content) > 0:
            input_ids = torch.tensor(tokenizer.encode(content, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors="np"))
            outputs = robert(input_ids)
            mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
            X_train.append(mean_pooling)
            y_train.append(label)
        else:
            X_train.append([0 for _ in range(768)])
            y_train.append(label)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return X_train, y_train


def preprocess_dataset(df, name):
    df = replace_unwanted_characters(df)
    df = normalize_text(df)
    
    text_embeddings, _ = get_titles_tokens(df)
    content_embeddings, _ = get_content_tokens(df)
    
    np.save(name+'title.npy', text_embeddings)
    np.save(name+'content.npy', content_embeddings)
    

In [ ]:
df = pd.read_csv('fourth.csv')
preprocess_dataset(df, 'fourth')

In [ ]:
# AICI ESTE MODELUL PENTRU TITLU SI CONTENT

In [15]:
class TitleAndContent(nn.Module):
    def __init__(self):
        super(TitleAndContent, self).__init__()
        self.linear_text = nn.Linear(768, 128)
        self.linear_content = nn.Linear(768, 128)
        self.linear_histogram = nn.Linear(18, 64)
        self.fc = nn.Linear(256 + 64, 256)
        self.bn = nn.BatchNorm1d(256)
        self.classifier = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, titles, contents, histograms):
        titles = self.linear_text(titles)
        contents = self.linear_content(contents)
        histograms = self.linear_histogram(histograms)

        titles = self.relu(titles)
        contents = self.relu(contents)
        histograms = self.relu(histograms)

        cat = torch.cat([titles, contents, histograms], dim=1)

        out = self.fc(cat)
        out = self.bn(out)
        out = self.relu(out)

        out = self.classifier(out)

        return self.sigmoid(out)

In [16]:
class TextAndContentDataset(Dataset):
    def __init__(self, x_titles, x_contents, x_hist, y):
        super(TextAndContentDataset, self).__init__()
        self.x_titles = x_titles
        self.x_contents = x_contents
        self.x_hist = x_hist
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, item):
        return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.x_hist[item]), torch.tensor(self.y[item])

In [17]:
def get_training_data_content():
    first = pd.read_csv('first.csv')
    first_indices = first[first['content'].str.len() > 1].index.tolist()
    first_titles = np.load('training_splits/firsttitle.npy')
    first_titles = first_titles[first_indices]
    first_contents = np.load('training_splits/firstcontent.npy')
    first_contents = first_contents[first_indices]
    first_labels = np.array(first['class'].tolist())
    first_labels = first_labels[first_indices]

    second = pd.read_csv('second.csv')
    second_indices = second[second['content'].str.len() > 1].index.tolist()
    second_titles = np.load('training_splits/secondtitle.npy')
    second_titles = second_titles[second_indices]
    second_contents = np.load('training_splits/secondcontent.npy')
    second_contents = second_contents[second_indices]
    second_labels = np.array(second['class'].tolist())
    second_labels = second_labels[second_indices]

    third = pd.read_csv('third.csv')
    third_indices = third[third['content'].str.len() > 1].index.tolist()
    third_titles = np.load('training_splits/thirdtitle.npy')
    third_titles = third_titles[third_indices]
    third_contents = np.load('training_splits/thirdcontent.npy')
    third_contents = third_contents[third_indices]
    third_labels = np.array(third['class'].tolist())
    third_labels = third_labels[third_indices]

    fourth = pd.read_csv('fourth.csv')
    fourth_indices = fourth[fourth['content'].str.len() > 1].index.tolist()
    fourth_titles = np.load('training_splits/fourthtitle.npy')
    fourth_titles = fourth_titles[fourth_indices]
    fourth_contens = np.load('training_splits/fourthcontent.npy')
    fourth_contens = fourth_contens[fourth_indices]
    fourth_labels = np.array(fourth['class'].tolist())
    fourth_labels = fourth_labels[fourth_indices]

    histograms = np.load('training_histograms.npy')

    all_titles = np.concatenate((first_titles, second_titles, third_titles, fourth_titles), axis=0)
    all_contents = np.concatenate((first_contents, second_contents, third_contents, fourth_contens), axis=0)
    all_labels = np.concatenate((first_labels, second_labels, third_labels, fourth_labels))
    return all_titles, all_contents, all_labels, histograms

def get_validation_data_content():
    fourth = pd.read_csv('fourth.csv')
    fourth_indices = fourth[fourth['content'].str.len() > 1].index.tolist()
    fourth_titles = np.load('training_splits/fourthtitle.npy')
    fourth_titles = fourth_titles[fourth_indices]
    fourth_contens = np.load('training_splits/fourthcontent.npy')
    fourth_contens = fourth_contens[fourth_indices]
    fourth_labels = np.array(fourth['class'].tolist())
    fourth_labels = fourth_labels[fourth_indices]

    return fourth_titles, fourth_contens, fourth_labels

In [19]:
X_train_titles, X_train_contents, y_train , X_train_hist= get_training_data_content()
# X_val_titles, X_val_contents, y_val = get_validation_data_content()

In [20]:
X_train_hist.shape

(69139, 18)

In [50]:
train_set = TextAndContentDataset(X_train_titles, X_train_contents,X_train_hist, y_train)
train_loader = DataLoader(train_set, shuffle=True, batch_size=2048)

# val_set = TextAndContentDataset(X_val_titles, X_val_contents,y_val)
# val_loader = DataLoader(val_set, shuffle=True, batch_size=16)

In [51]:
model_text_and_content = TitleAndContent().to(device)
optimizer = optim.Adam(model_text_and_content.parameters(),lr=1e-4)
criterion = nn.BCELoss(weight=torch.tensor([len(y_train) / np.sum(y_train)]).to(device))

In [52]:
EPOCHS = 40

In [53]:
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm

for epoch in range(EPOCHS):
    running_loss = 0
    running_acc = 0

    train_pred_labels = []
    train_true_labels = []

    for X_title, X_content,X_hist, y in tqdm(train_loader):
        y = y.to(device, dtype=torch.long)
        X_title = X_title.to(device, dtype=torch.float)
        X_hist = X_hist.to(device, dtype=torch.float)
        X_content = X_content.to(device, dtype=torch.float)

        output = model_text_and_content(X_title, X_content, X_hist)

        output.squeeze(1)
        pred = output > 0.5

        train_pred_labels.extend(pred.cpu().numpy().tolist())
        train_true_labels.extend(y.cpu().numpy().tolist())

        loss = criterion(output.squeeze(1), y.float())
        running_loss += loss.item()

        model_text_and_content.zero_grad()
        loss.backward()
        optimizer.step()

    # pred_labels = []
    # true_labels = []
    # 
    # model_text_and_content.eval()
    # val_loss = 0
    # with torch.no_grad():
    #     for X_title, X_content, y in tqdm(val_loader):
    #         y = y.to(device, dtype=torch.long)
    #         X_title = X_title.to(device, dtype=torch.float)
    #         X_content = X_content.to(device, dtype=torch.float)
    # 
    #         output = model_text_and_content(X_title, X_content)
    #         val_loss += criterion(output.squeeze(1), y.float()).item()
    #         output.squeeze(1)
    # 
    #         pred = (output > 0.5)
    #         pred_labels.extend(pred.cpu().numpy().tolist())
    #         true_labels.extend(y.cpu().numpy().tolist())
    # 
    # model_text_and_content.train()
    # 
    # precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='macro')
    precision_train, recall_train, f1_train, _ = precision_recall_fscore_support(train_pred_labels, train_true_labels, average='macro')


    # print(f"Epoch: {epoch + 1} | Train Loss {running_loss / len(train_set)}  | Train Precision: {precision_train} | Train Recall: {recall_train} | Train F1: {f1_train} | Val Loss: {val_loss / len(val_set)} | Val Precision: {precision} | Val Recall: {recall} | Val F1: {f1}")
    print(f"Epoch: {epoch + 1} | Train Loss {running_loss / len(train_set)}  | Train Precision: {precision_train} | Train Recall: {recall_train} | Train F1: {f1_train}")

torch.save(model_text_and_content.state_dict(), 'models/titleAndContentClassifier.pth')

  0%|          | 0/34 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_11296\2085392579.py:13: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.x_hist[item]), torch.tensor(self.y[item])
100%|██████████| 34/34 [00:01<00:00, 20.95it/s]


Epoch: 1 | Train Loss 0.00029485829060716615  | Train Precision: 0.8631395968847366 | Train Recall: 0.8366460433078468 | Train F1: 0.8373539660349287


  0%|          | 0/34 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_11296\2085392579.py:13: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.x_hist[item]), torch.tensor(self.y[item])
100%|██████████| 34/34 [00:01<00:00, 21.25it/s]


Epoch: 2 | Train Loss 0.00012831705381192423  | Train Precision: 0.9632127482805068 | Train Recall: 0.9591557740407921 | Train F1: 0.9611136741348572


  0%|          | 0/34 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_11296\2085392579.py:13: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.x_hist[item]), torch.tensor(self.y[item])
100%|██████████| 34/34 [00:01<00:00, 23.83it/s]


...

Epoch: 38 | Train Loss 6.243098813637805e-06  | Train Precision: 0.9988297148002159 | Train Recall: 0.9990765590161771 | Train F1: 0.9989529164701917


  0%|          | 0/34 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_11296\2085392579.py:13: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.x_hist[item]), torch.tensor(self.y[item])
100%|██████████| 34/34 [00:03<00:00,  8.89it/s]


Epoch: 39 | Train Loss 5.923395694505696e-06  | Train Precision: 0.9990057958728571 | Train Recall: 0.9991504270233822 | Train F1: 0.9990780357085048


  0%|          | 0/34 [00:00<?, ?it/s]C:\Users\Alex\AppData\Local\Temp\ipykernel_11296\2085392579.py:13: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.tensor(self.x_titles[item]), torch.tensor(self.x_contents[item]), torch.tensor(self.x_hist[item]), torch.tensor(self.y[item])
100%|██████████| 34/34 [00:03<00:00,  9.03it/s]


Epoch: 40 | Train Loss 5.42647661713444e-06  | Train Precision: 0.9990512638520328 | Train Recall: 0.9992299922488117 | Train F1: 0.9991405124559314


In [54]:
# AICI SE TERMINA MODELUL PENTRU TITLU SI CONTENT
# DE AICI INCEPE PENTRU SUBMISIE

In [55]:
def get_all_test_data():
    file_names = ['first', 'second', 'third', 'fourth', 'fifth', 'sixth']
    
    all_titles = np.array([])
    all_contents = np.array([])
    
    for f in file_names:
        titles = np.load(f'testing_splits/{f}_testtitle.npy')
        content = np.load(f'testing_splits/{f}_testcontent.npy')
        if len(all_titles) == 0:
            all_titles = titles
            all_contents = content
        else:
            all_titles = np.concatenate((all_titles, titles), axis=0)
            all_contents = np.concatenate((all_contents, content), axis=0)
            
    return all_titles, all_contents

In [56]:
X, y = get_all_test_data()
df = pd.read_csv('test.csv')
testing_hist = np.load('testing_histograms.npy')

In [57]:
all_pred = []
ids = []

# model_titles = ClssificationNetwork().to(device)
# model_titles.load_state_dict(torch.load('haisavedem.pth'))

model_titles.eval()
model_text_and_content.eval()

indices = df[df['content'].str.len() > 1].index.tolist()
cnt = 0
for i in tqdm(range(len(df))):
    title = X[i]
    content = y[i]
    if i in indices:
        title = torch.tensor(title, dtype=torch.float).to(device).unsqueeze(0)
        content = torch.tensor(content, dtype=torch.float).to(device).unsqueeze(0)
        hist = torch.tensor(testing_hist[cnt], dtype=torch.float).to(device).unsqueeze(0)
        cnt += 1
        output = model_text_and_content(title, content, hist)
        output.squeeze(1)
        pred = (output > 0.5).item()
    else:
        title = torch.tensor(title, dtype=torch.float).to(device).unsqueeze(0)
        output = model_titles(title)
        output.squeeze(1)
        pred = (output > 0.5).item()
        
    all_pred.append(pred)
    ids.append(i)

100%|██████████| 36669/36669 [00:23<00:00, 1540.84it/s]


In [58]:
submisie = pd.DataFrame({'id':ids, 'class':list(map(int, all_pred))})
submisie.to_csv('submission_embeddings_hist_batch_norm_20epochs_2048batch.csv', index=False)

In [ ]:
# GATAAAAAAAAAAAA

In [ ]:
def get_titles_tokens(df):
    X_train = []
    aux = df['title'].tolist()

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})

    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))

    for title in tqdm(aux):
        input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
        outputs = robert(input_ids)
        mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
        X_train.append(mean_pooling)

    X_train = np.array(X_train)

    return X_train

def get_content_tokens(df):
    X_train = []
    y_train = []
    aux = df['content'].tolist()

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})

    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))

    for content in tqdm(aux):
        if len(content) > 0:
            input_ids = torch.tensor(tokenizer.encode(content, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors="np"))
            outputs = robert(input_ids)
            mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
            X_train.append(mean_pooling)
        else:
            X_train.append([0 for _ in range(768)])

    X_train = np.array(X_train)

    return X_train


def preprocess_dataset(df, name):
    df = replace_unwanted_characters(df)
    df = normalize_text(df)

    text_embeddings = get_titles_tokens(df)
    content_embeddings = get_content_tokens(df)

    np.save(name+'title.npy', text_embeddings)
    np.save(name+'content.npy', content_embeddings)

In [ ]:
test_df = pd.read_csv('first_test.csv')
preprocess_dataset(test_df, 'first_test')

In [ ]:
test_df = pd.read_csv('sixth_test.csv')
preprocess_dataset(test_df, 'sixth_test')